In [40]:
import pandas as pd
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from scipy.sparse import lil_matrix
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [41]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [42]:
data = pd.read_csv('2month_new.csv', index_col=0)

In [44]:
data.head()

,mro_new,est_hh_incm_prmr_cd,purchaser_age_at_tm_of_purch,input_indiv_gndr_prmr_cd,gmqualty_model,umf_xref_finc_gbl_trim,engn_size,purchase_time,hard_braking_0,hard_acceleration_0,speeding_sum_0,day_mileage_0,tavg_0,random_avg_traffic_0,mro_new_indicator_1,hard_braking_1,hard_acceleration_1,speeding_sum_1,day_mileage_1,tavg_1,random_avg_traffic_1
1,0.0,6.0,54.0,0,13,0,2.0,0,45.666667,4.666667,23.666667,325.572917,15.110922,0.240708,0.0,33.333333,5.666667,19.333333,298.156250,15.123979,0.245754
2,0.0,6.0,54.0,0,13,0,2.0,0,51.333333,7.666667,29.000000,288.062500,15.106969,0.238053,0.0,45.666667,4.666667,23.666667,325.572917,15.110922,0.240708
3,0.0,6.0,54.0,0,13,0,2.0,0,51.666667,11.000000,28.000000,252.713542,15.050637,0.233105,0.0,51.333333,7.666667,29.000000,288.062500,15.106969,0.238053
4,0.0,6.0,54.0,0,13,0,2.0,0,68.500000,13.500000,36.500000,507.960935,15.497578,0.227600,0.0,51.666667,11.000000,28.000000,252.713542,15.050637,0.233105
5,1.0,6.0,54.0,0,13,0,2.0,0,64.333333,10.666667,27.000000,371.135417,15.588944,0.225047,0.0,68.500000,13.500000,36.500000,507.960935,15.497578,0.227600


In [5]:
y = data['mro_new']
X = data.iloc[:, 1:]
y = y.astype(int)
X = X.astype(float)
X = StandardScaler().fit_transform(X)
X = np.c_[X]

In [48]:
data.corr()

,mro_new,est_hh_incm_prmr_cd,purchaser_age_at_tm_of_purch,input_indiv_gndr_prmr_cd,gmqualty_model,umf_xref_finc_gbl_trim,engn_size,purchase_time,hard_braking_0,hard_acceleration_0,speeding_sum_0,day_mileage_0,tavg_0,random_avg_traffic_0,mro_new_indicator_1,hard_braking_1,hard_acceleration_1,speeding_sum_1,day_mileage_1,tavg_1,random_avg_traffic_1
mro_new,1.000000,-0.002522,-0.008472,0.011919,0.014274,0.018670,0.030220,0.022113,-0.001828,-0.003748,0.000793,0.060063,0.003866,-0.007592,-0.093205,0.000268,-0.003142,0.000797,0.060573,0.010626,-0.005052
est_hh_incm_prmr_cd,-0.002522,1.000000,0.052070,0.076911,0.009798,0.049412,0.018772,-0.025074,-0.007245,0.051529,0.017152,-0.013962,-0.008158,-0.037798,0.000615,-0.008269,0.050249,0.016376,-0.015091,-0.008284,-0.035723
purchaser_age_at_tm_of_purch,-0.008472,0.052070,1.000000,0.022464,-0.082045,-0.067322,-0.140719,0.018131,-0.054218,-0.079702,-0.146115,-0.109195,-0.000650,-0.054808,-0.006139,-0.053965,-0.079415,-0.146414,-0.108057,-0.000958,-0.055651
input_indiv_gndr_prmr_cd,0.011919,0.076911,0.022464,1.000000,0.188669,0.286508,0.381932,-0.059930,-0.146103,-0.018466,-0.094852,0.046006,-0.000532,0.035750,0.017212,-0.144448,-0.017141,-0.094118,0.045819,-0.003743,0.037472
gmqualty_model,0.014274,0.009798,-0.082045,0.188669,1.000000,0.362879,0.432391,0.005323,-0.121003,-0.018364,-0.057773,0.053152,0.004151,0.103486,0.019114,-0.118909,-0.018139,-0.055620,0.053317,0.002681,0.104195
umf_xref_finc_gbl_trim,0.018670,0.049412,-0.067322,0.286508,0.362879,1.000000,0.651060,-0.273040,-0.203793,-0.088247,-0.084412,0.057267,0.008958,0.148098,0.027135,-0.201995,-0.083492,-0.083854,0.057359,0.000426,0.151768
engn_size,0.030220,0.018772,-0.140719,0.381932,0.432391,0.651060,1.000000,-0.156022,-0.256308,-0.061495,-0.146202,0.095470,0.005501,0.186362,0.040891,-0.253360,-0.057959,-0.144473,0.095600,-0.000894,0.188939
purchase_time,0.022113,-0.025074,0.018131,-0.059930,0.005323,-0.273040,-0.156022,1.000000,0.056198,0.033422,-0.022407,0.074450,0.031041,-0.113547,0.015967,0.059631,0.035323,-0.016582,0.080873,0.081879,-0.122291
hard_braking_0,-0.001828,-0.007245,-0.054218,-0.146103,-0.121003,-0.203793,-0.256308,0.056198,1.000000,0.317918,0.774196,0.243887,0.037094,0.031903,-0.015328,0.783736,0.214828,0.596874,0.130747,0.034409,0.028744
hard_acceleration_0,-0.003748,0.051529,-0.079702,-0.018466,-0.018364,-0.088247,-0.061495,0.033422,0.317918,1.000000,0.414653,-0.006178,0.004977,-0.000194,-0.010920,0.228316,0.766318,0.306444,-0.072930,0.006028,-0.003790


### four types

In [6]:
num_neighbor=5
knn = NearestNeighbors(n_neighbors=num_neighbor, metric='euclidean', n_jobs=-1)
knn.fit(X) 
distances, indices = knn.kneighbors(X)
count_neighbor = np.zeros((X.shape[0], num_neighbor + 2))

In [7]:
y = y.to_numpy()
for i in range(X.shape[0]):
    count_neighbor[i, :num_neighbor] = y[indices[i, :]]
    count_neighbor[i, num_neighbor] = abs(5 * y[i] - np.sum(count_neighbor[i, :num_neighbor]))

    if count_neighbor[i, num_neighbor] == 5:
        count_neighbor[i, num_neighbor + 1] = 0.1
    elif count_neighbor[i, num_neighbor] == 4:
        count_neighbor[i, num_neighbor + 1] = 0.25
    elif count_neighbor[i, num_neighbor] == 3:
        count_neighbor[i, num_neighbor + 1] = 0.4
    elif count_neighbor[i, num_neighbor] == 2:
        count_neighbor[i, num_neighbor + 1] = 0.6
    elif count_neighbor[i, num_neighbor] == 1:
        count_neighbor[i, num_neighbor + 1] = 0.8
    else:
        count_neighbor[i, num_neighbor + 1] = 1
X_all = data
X_all['degree'] = count_neighbor[:, num_neighbor + 1]

### LOF

In [8]:
ir = len(y[y == 1]) / len(y)
clf = LocalOutlierFactor(n_neighbors=10, contamination=ir)  # highly rely on the IR in training set
y_pred = clf.fit_predict(X)
X_scores = clf.negative_outlier_factor_
X_scores_1 = np.array([X_scores[x:x + 1] for x in range(0, len(X_scores), 1)])  # change shape (n,) to (n, 1)
X_all['LOF'] = X_scores_1

In [9]:
X_all.to_csv('month.csv')

### run

In [29]:
X_all = pd.read_csv('week.csv', index_col=0)

In [38]:
X_new = X_all.drop(['mro_new'], axis=1)
y = X_all['mro_new']
X_new = X_new[ X_new.columns.drop(list( X_new.filter(regex='hard|speed')))]
X_add = StandardScaler().fit_transform(X_new)
X_add = np.c_[X_add]

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_add, y, test_size=0.2, random_state=42)
clf = lgb.LGBMClassifier(random_state=42, n_estimators=100, learning_rate=0.1, class_weight='balanced', force_col_wise=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

[LightGBM] [Info] Number of positive: 88474, number of negative: 1991878
[LightGBM] [Info] Total Bins 6547
[LightGBM] [Info] Number of data points in the train set: 2080352, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Precision: 0.9721
Recall: 0.9924
F1-score: 0.9821
